In [1]:
import sys

sys.path.append("..")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import h5py
import timm
import torch
import torch.nn as nn
from sklearn.model_selection import StratifiedKFold
from timm import create_model
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
from transformers.optimization import (
    get_linear_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
)
import torch.nn.functional as F
from pathlib import Path
from torch.utils.data import DataLoader
import os
from fastprogress.fastprogress import master_bar, progress_bar
from fastai.vision.all import L, unsqueeze
from timm.data.mixup import Mixup
from timm.loss import (
    LabelSmoothingCrossEntropy,
    BinaryCrossEntropy,
    SoftTargetCrossEntropy,
)
import random
from einops import rearrange

In [2]:
def get_snr(left, right, df):
    df_ = pd.concat([df.query(f"snr>{left} & snr<{right}"), df.query("snr==0")])
    return df_


def generate_report(df, p, fn):
    pred = F.softmax(p).cpu().numpy()[:, 1]
    val_df_eval = df.copy()
    val_df_eval["pred"] = pred
    val_df_eval.to_csv(f'{fn}_oof.csv')

    roc_100 = roc_auc_score(val_df_eval["target"], val_df_eval["pred"])
    roc_50_100 = roc_auc_score(
        get_snr(0, 50, val_df_eval)["target"], get_snr(0, 50, val_df_eval)["pred"]
    )
    roc_0_50 = roc_auc_score(
        get_snr(0, 50, val_df_eval)["target"], get_snr(0, 50, val_df_eval)["pred"]
    )
    roc_0_40 = roc_auc_score(
        get_snr(0, 40, val_df_eval)["target"], get_snr(0, 40, val_df_eval)["pred"]
    )
    roc_0_30 = roc_auc_score(
        get_snr(0, 30, val_df_eval)["target"], get_snr(0, 30, val_df_eval)["pred"]
    )

    roc_0_20 = roc_auc_score(
        get_snr(0, 20, val_df_eval)["target"], get_snr(0, 20, val_df_eval)["pred"]
    )
    


    return {
        "roc_all": roc_100,
        "roc_50_100": roc_50_100,
        "roc_0_50": roc_0_50,
        "roc_0_40": roc_0_40,
        "roc_0_30": roc_0_30,
        "roc_0_20": roc_0_20,
    }

def normalize(x, pmin=3, pmax=97, axis=None, clip=False, eps=1e-20, dtype=np.float32):
    """Percentile-based image normalization."""

    mi = np.percentile(x,pmin,axis=axis,keepdims=True)
    ma = np.percentile(x,pmax,axis=axis,keepdims=True)
    return normalize_mi_ma(x, mi, ma, clip=clip, eps=eps, dtype=dtype)


def normalize_mi_ma(x, mi, ma, clip=False, eps=1e-20, dtype=np.float32):
    if dtype is not None:
        x   = x.astype(dtype,copy=False)
        mi  = dtype(mi) if np.isscalar(mi) else mi.astype(dtype,copy=False)
        ma  = dtype(ma) if np.isscalar(ma) else ma.astype(dtype,copy=False)
        eps = dtype(eps)

    try:
        import numexpr
        x = numexpr.evaluate("(x - mi) / ( ma - mi + eps )")
    except ImportError:
        x =                   (x - mi) / ( ma - mi + eps )

    if clip:
        x = np.clip(x,0,1)

    return x


In [3]:
class SaveModel:
    def __init__(self, folder, exp_name, best=np.inf):
        self.best = best
        self.folder = Path(folder) / f"{exp_name}.pth"

    def __call__(self, score, model, epoch):
        if score < self.best:
            self.best = score
            print(f"Better model found at epoch {epoch} with value: {self.best}.")
            torch.save(model.state_dict(), self.folder)


class SaveModelMetric:
    def __init__(self, folder, exp_name, best=-np.inf):
        self.best = best
        self.folder = Path(folder) / f"{exp_name}.pth"

    def __call__(self, score, model, epoch):
        if score > self.best:
            self.best = score
            print(f"Better model found at epoch {epoch} with value: {self.best}.")
            torch.save(model.state_dict(), self.folder)


class SaveModelEpoch:
    def __init__(self, folder, exp_name, best=-np.inf):
        self.best = best
        self.folder = Path(folder)
        self.exp_name = exp_name

    def __call__(self, score, model, epoch):
        self.best = score
        print(f"Better model found at epoch {epoch} with value: {self.best}.")
        torch.save(model.state_dict(), f"{self.folder/self.exp_name}_{epoch}.pth")


def custom_auc_score(p, gt):
    return roc_auc_score(gt.cpu().numpy(), F.softmax(p).cpu().numpy()[:, 1])


def fit_mixup(
    epochs,
    model,
    train_dl,
    valid_dl,
    loss_fn,
    opt,
    metric,
    val_df,
    folder="models",
    exp_name="exp_00",
    device=None,
    sched=None,
    mixup_=False,
    save_md=SaveModelEpoch,
):
    if device is None:
        device = (
            torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        )

    os.makedirs(folder, exist_ok=True)
    loss_fn_trn = loss_fn
    if mixup_:
        mixup = Mixup(
            num_classes=2, mixup_alpha=0.4, prob=0.8
        )
        loss_fn_trn = BinaryCrossEntropy()
    mb = master_bar(range(epochs))
    
   
    mb.write(["epoch", 
              "train_loss",
              "valid_loss",
              "val_metric",
              "roc_all",
              "roc_50_100", 
              "roc_0_50", 
              "roc_0_40", 
              "roc_0_30", 
              "roc_0_20"], table=True)
    model.to(device)  # we have to put our model on gpu
    scaler = torch.cuda.amp.GradScaler()  # this for half precision training
    save_md = save_md(folder, exp_name)

    for i in mb:  # iterating  epoch
        trn_loss, val_loss = 0.0, 0.0
        trn_n, val_n = len(train_dl.dataset), len(valid_dl.dataset)
        model.train()  # set model for training
        for (xb, yb) in progress_bar(train_dl, parent=mb):
            xb, yb = xb.to(device), yb.to(device)  # putting batches to device
            if mixup_:
                xb, yb = mixup(xb, yb)
            with torch.cuda.amp.autocast():  # half precision
                out = model(xb)  # forward pass
                loss = loss_fn_trn(out, yb)  # calulation loss

            trn_loss += loss.item()
            opt.zero_grad()  # zeroing optimizer
            scaler.scale(loss).backward()  # backward
            scaler.step(opt)  # optimzers step
            scaler.update()  # for half precision
            if sched is not None:
                sched.step()  # scuedular step

        trn_loss /= mb.child.total

        # putting model in eval mode
        model.eval()
        gt = []
        pred = []
        # after epooch is done we can run a validation dataloder and see how are doing
        with torch.no_grad():
            for (xb, yb) in progress_bar(valid_dl, parent=mb):
                xb, yb = xb.to(device), yb.to(device)
                out = model(xb)
                loss = loss_fn(out, yb)
                val_loss += loss.item()

                gt.append(yb.detach())
                pred.append(out.detach())
        # calculating metric
        metric_ = metric(torch.cat(pred), torch.cat(gt))
        # saving model if necessary
        save_md(metric_, model, i)
        val_loss /= mb.child.total
        dict_res = generate_report(val_df, torch.cat(pred), f"{folder}/{exp_name}_{i}")
            
        pd.DataFrame(
            {
                "trn_loss": [trn_loss],
                "val_loss": [val_loss],
                "metric": [metric_],
                "roc_all": [dict_res["roc_all"]],
                "roc_50_100": [dict_res["roc_50_100"]],
                "roc_0_50": [dict_res["roc_0_50"]],
                "roc_0_40": [dict_res["roc_0_40"]],
                "roc_0_30": [dict_res["roc_0_30"]],
                "roc_0_20": [dict_res["roc_0_20"]],
            }
        ).to_csv(f"{folder}/{exp_name}_{i}.csv", index=False)
        mb.write(
            [
                i,
                f"{trn_loss:.6f}",
                f"{val_loss:.6f}",
                f"{metric_:.6f}",
                f"{dict_res['roc_all']:.6f}",
                f"{dict_res['roc_50_100']:.6f}",
                f"{dict_res['roc_0_50']:.6f}",
                f"{dict_res['roc_0_40']:.6f}",
                f"{dict_res['roc_0_30']:.6f}",
                f"{dict_res['roc_0_20']:.6f}",
            ],
            table=True,
        )
    print("Training done")
    # loading the best checkpoint

In [4]:
class DataV0:
    """
    dataset = Dataset(data_type, df)

    img, y = dataset[i]
      img (np.float32): 2 x 360 x 128
      y (np.float32): label 0 or 1
    """

    def __init__(self, df, tfms=False):
        self.df = df
        self.tfms = tfms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        """
        i (int): get ith data
        """
        r = self.df.iloc[i]
        y = np.float32(r.target)
        filename=r.id
        img = np.array(torch.load(filename)['s_p_n'])

        if self.tfms:
            if np.random.rand() <= 0.5:  # horizontal flip
                img = np.flip(img, axis=1).copy()
            if np.random.rand() <= 0.5:  # vertical flip
                img = np.flip(img, axis=2).copy()
            if np.random.rand() <= 0.5:  # vertical shift
                img = np.roll(img, np.random.randint(low=0, high=img.shape[1]), axis=1)

        return img,  y.astype('int')

In [28]:
class CFG:
    bs = 64
    nw = 4
    model_name = "convnext_xlarge_384_in22ft1k"
    lr = 1e-4
    wd = 1e-4
    epoch = 10
    warmup_pct = 0.1
    num_classes = 2
    dropout_rate = 0.3
    folder = "EXP_40_00_LARGE_KERMEL"
    mixup=False
    split_voldf = Path("../data/SPLITS/V_20")
    exp_name = f"{folder}_{model_name}_{split_voldf.stem}_{mixup}"

In [29]:
dforig = pd.read_csv('../data/train_labels.csv')
dforig.columns = ['fn', 'target']
dforig['fn'] = dforig['fn'].apply(lambda x: Path('../data/train')/f'{x}.hdf5')
dforig.columns = ['id', 'target']
dforig['id'] = dforig['id'].apply(lambda x: str(x).replace('.hdf5', '.pth'))
dforig = dforig[dforig.target >= 0].reset_index(drop=True)


trn_df = pd.read_csv(CFG.split_voldf/'trn_df.csv')
trn_df['id'] = trn_df['id'].apply(lambda x: x.replace('.h5', '.pth'))

val_df = pd.read_csv(CFG.split_voldf/'val_df.csv')
val_df['id'] = val_df['id'].apply(lambda x: x.replace('.h5', '.pth'))
trn_df = pd.concat([dforig, trn_df]).sample(frac=1)
trn_df.shape,  val_df.shape

((15642, 13), (2400, 13))

In [30]:
dforig['target'].value_counts()

1    400
0    200
Name: target, dtype: int64

In [34]:
custom_model = create_model(
    CFG.model_name,
    pretrained=True,
    num_classes=CFG.num_classes,
    in_chans=16,
)
C, _,H, W = custom_model.stem[0].weight.shape

In [35]:
nn.Conv2d(2, C, [H, W], 1, [H//2, W//2], 1, 1, False)

Conv2d(2, 256, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2), bias=False)

In [8]:
gt = []
preds = []
fold = 0

# Train - val split
trn_ds = DataV0(trn_df, True)
vld_ds = DataV0(val_df)

trn_dl = DataLoader(
    trn_ds,
    batch_size=CFG.bs,
    shuffle=True,
    num_workers=CFG.nw,
    pin_memory=True,
    drop_last=True,
)
vld_dl = DataLoader(
    vld_ds,
    batch_size=CFG.bs,
    shuffle=False,
    num_workers=CFG.nw,
    pin_memory=True,
)


custom_model = create_model(
    CFG.model_name,
    pretrained=True,
    num_classes=CFG.num_classes,
    in_chans=2,
)

opt = torch.optim.AdamW(custom_model.parameters(), lr=CFG.lr, weight_decay=CFG.wd)
loss_func = nn.CrossEntropyLoss()
warmup_steps = int(len(trn_dl) * int(CFG.warmup_pct * CFG.epoch))
total_steps = int(len(trn_dl) * CFG.epoch)
sched = get_linear_schedule_with_warmup(
    opt, num_warmup_steps=warmup_steps, num_training_steps=total_steps
)

fit_mixup(
    epochs=CFG.epoch,
    model=custom_model,
    train_dl=trn_dl,
    valid_dl=vld_dl,
    loss_fn=loss_func,
    opt=opt,
    val_df = val_df,
    metric=custom_auc_score,
    folder=CFG.folder,
    exp_name=f"{CFG.exp_name}_{fold}",
    device="cuda:0",
    sched=sched,
)
del custom_model
del trn_dl
del vld_dl
import gc

gc.collect()

epoch,train_loss,valid_loss,val_metric,roc_all,roc_50_100,roc_0_50,roc_0_40,roc_0_30,roc_0_20
0,0.434529,0.716565,0.622311,0.622311,0.622311,0.622311,0.586791,0.587122,0.598719
1,0.393932,0.789931,0.663632,0.663632,0.663632,0.663632,0.618497,0.616471,0.627463
2,0.376774,0.696332,0.671189,0.671189,0.671189,0.671189,0.623251,0.614588,0.627677
3,0.366054,0.717532,0.682981,0.682981,0.682981,0.682981,0.629709,0.610745,0.618950
4,0.358006,0.658183,0.692357,0.692357,0.692357,0.692357,0.629936,0.596939,0.599141
5,0.354711,0.721797,0.682333,0.682333,0.682333,0.682333,0.618819,0.593952,0.605552
6,0.345064,0.686799,0.689731,0.689731,0.689731,0.689731,0.624305,0.590313,0.612366
7,0.338494,0.709532,0.692427,0.692427,0.692427,0.692427,0.632391,0.603588,0.618455
8,0.330864,0.724993,0.695580,0.695580,0.695580,0.695580,0.632732,0.595380,0.599371
9,0.324708,0.704319,0.692718,0.692718,0.692718,0.692718,0.629163,0.590880,0.602264


/tmp/ipykernel_605931/2678276579.py:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return roc_auc_score(gt.cpu().numpy(), F.softmax(p).cpu().numpy()[:, 1])


Better model found at epoch 0 with value: 0.6223114583333332.


/tmp/ipykernel_605931/2109093364.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = F.softmax(p).cpu().numpy()[:, 1]
/tmp/ipykernel_605931/2678276579.py:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return roc_auc_score(gt.cpu().numpy(), F.softmax(p).cpu().numpy()[:, 1])


Better model found at epoch 1 with value: 0.6636322916666667.


...

/tmp/ipykernel_605931/2109093364.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = F.softmax(p).cpu().numpy()[:, 1]
/tmp/ipykernel_605931/2678276579.py:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return roc_auc_score(gt.cpu().numpy(), F.softmax(p).cpu().numpy()[:, 1])


Better model found at epoch 9 with value: 0.6927180555555555.


/tmp/ipykernel_605931/2109093364.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = F.softmax(p).cpu().numpy()[:, 1]


Training done


88